In [18]:
import pandas as pd
import numpy as np
import pickle

In [19]:
#url = 'https://raw.githubusercontent.com/MariaRosendal/Enhancing-Price-Momentum-with-RNN/main/'
url= ""

In [20]:
# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
#seed_value= 123
#import os
#os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
#import random
#random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
#import numpy as np
#np.random.seed(seed_value)

# 4. Set `tensorflow` pseudo-random generator at a fixed value
#import tensorflow as tf
#tf.random.set_seed(seed_value)

# 5. For layers that introduce randomness like dropout, make sure to set seed values 
#model.add(Dropout(0.25, seed=seed_value))
#6 Configure a new global `tensorflow` session
#from keras import backend as K
#session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
#sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
#K.set_session(sess)

## 1. Importing input and output variables

In [6]:
# Import X 
X_train = pickle.load(open('X_train_scaled.pkl', 'rb' ))
X_test = pickle.load(open('X_test_scaled.pkl', 'rb' ))

In [7]:
# Import X for MLP and RF 
#X_train_mlp = pickle.load(open('X_train_vec.pkl', 'rb' ))
#X_test_mlp = pickle.load(open('X_test_vec.pkl', 'rb' ))

In [8]:
# Checking for NaN values
print(np.count_nonzero(np.isnan(X_train)), np.count_nonzero(np.isnan(X_test)))

0 0


In [9]:
X_train.shape, X_test.shape, 

((270000, 12, 9), (270000, 12, 9))

In [10]:
# Importing y
y = pd.read_csv(url+'y_reg.csv')
y['date'] =  pd.to_datetime(y['date'])
y = y.set_index(['date'])

In [11]:
# Defining length of y_train and y_test
y_train = y[y.index <= '1975-12-01']
y_test = y[y.index > '1975-12-01']

In [12]:
y_train.shape, y_test.shape

((270000, 1), (270000, 1))

In [13]:
# Reformating y
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()
y_train.shape, y_test.shape

((270000,), (270000,))

In [15]:
y_train

array([ 0.134864,  0.201948,  0.036379, ..., -0.072761, -0.051312,
        0.030733])

## 2. Models

In [13]:
# Importing packages
import tensorflow as tf
import matplotlib.pyplot as plt

import math
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import BatchNormalization 
from keras.layers.core import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping

from sklearn.metrics import mean_squared_error

from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import r2_score

### Linear Regression

In [15]:
# Reshaping X to vectors
n_input = X_train.shape[1] * X_train.shape[2]
X_train_mlp = X_train.reshape(len(X_train),n_input)
X_test_mlp = X_test.reshape(len(X_test),n_input)

In [14]:
from sklearn.linear_model import LinearRegression

In [16]:
m_lin = LinearRegression()
m_lin.fit(X_train_mlp, y_train)

LinearRegression()

In [17]:
lin_predict = m_lin.predict(X_test_mlp)

In [18]:
testScore = math.sqrt(mean_squared_error(y_test, lin_predict))
print('Test Score: %.2f RMSE' % (testScore))
print(testScore)
print((r2_score(y_test, lin_predict)))

Test Score: 0.00 RMSE
0.0009594408586506351
-0.057806861150169864


In [19]:
lin = pd.DataFrame(lin_predict)
lin.nunique()

0    270000
dtype: int64

In [20]:
pickle.dump(lin_predict, open('lin_predict.pkl', 'wb' ))

### Random Forest

In [16]:
X_train_mlp.shape,  X_test_mlp.shape

((270000, 108), (270000, 108))

In [17]:
from skopt.space import Real, Integer
from skopt import BayesSearchCV

In [23]:
# Ensuring data keeps temporal element when split
tscv = TimeSeriesSplit(n_splits=9) 

In [24]:
from sklearn.ensemble import RandomForestRegressor
import sklearn

In [25]:
m_rf = RandomForestRegressor()

In [26]:
n_estimators = 300

In [28]:
#parameters_rf = {'n_estimators':[50, 100, 200, 300], 'max_depth':[3, 5, 7], 'min_samples_split':[3, 5, 10, 50]}
param_opt = dict(max_depth=Integer(1,10), max_features=Integer(3,108))

In [29]:
bayesian = BayesSearchCV(
     m_rf,
     param_opt,
     n_iter=32,
     random_state=100,
     cv=tscv,
     verbose = 0, n_jobs=1
 )

bayesian_result = bayesian.fit(X_train_mlp, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


In [43]:
# Best model parameters
print('According to BayesianOptimization the best parameters are : ')
print('max_depth : ' + str(bayesian_result.best_params_['max_depth']))
print('max_features : ' + str(bayesian_result.best_params_['max_features']))

According to BayesianOptimization the best parameters are : 
max_depth : 1
max_features : 3


In [31]:
# Selecting best model
print("Best: %f using %s" % (bayesian_result.best_score_, bayesian_result.best_params_))
means = bayesian_result.cv_results_['mean_test_score']
stds = bayesian_result.cv_results_['std_test_score']
params = bayesian_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.002944 using OrderedDict([('max_depth', 1), ('max_features', 3)])
-0.021947 (0.034739) with: OrderedDict([('max_depth', 5), ('max_features', 21)])
-0.026370 (0.045516) with: OrderedDict([('max_depth', 4), ('max_features', 46)])
-0.030857 (0.055087) with: OrderedDict([('max_depth', 5), ('max_features', 60)])
-0.027277 (0.042697) with: OrderedDict([('max_depth', 8), ('max_features', 36)])
-0.038808 (0.061843) with: OrderedDict([('max_depth', 5), ('max_features', 77)])
-0.031985 (0.053647) with: OrderedDict([('max_depth', 2), ('max_features', 77)])
-0.044281 (0.064545) with: OrderedDict([('max_depth', 7), ('max_features', 72)])
-0.017745 (0.025772) with: OrderedDict([('max_depth', 6), ('max_features', 10)])
-0.026307 (0.045355) with: OrderedDict([('max_depth', 6), ('max_features', 53)])
-0.035861 (0.048723) with: OrderedDict([('max_depth', 9), ('max_features', 26)])
-0.007457 (0.010530) with: OrderedDict([('max_depth', 2), ('max_features', 8)])
-0.029266 (0.068707) with: OrderedD

In [32]:
# Defining best model parameters
max_depth = bayesian_result.best_params_['max_depth']
max_features = bayesian_result.best_params_['max_features']

In [33]:
#best_rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features)
best_rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, max_features=max_depth)

best_rf.fit(X_train_mlp, y_train)

RandomForestRegressor(max_depth=1, max_features=1, n_estimators=300)

In [34]:
print("Random forest:\t\t\t" + str(best_rf.score(X_test_mlp, y_test)))

Random forest:			-0.0001377252949832375


In [35]:
rf_predict = best_rf.predict(X_test_mlp)

In [36]:
print(r2_score(y_test, rf_predict))

-0.0001377252949832375


In [46]:
r2_df = pd.DataFrame()
r2_df['y'] = y_test
r2_df['y_hat'] = rf_predict
r2_df['difference'] = (r2_df['y']-r2_df['y_hat'])

In [47]:
r2_df = pd.DataFrame()
r2_df['y'] = y_test
r2_df['y_hat'] = rf_predict
r2_df['difference'] = (r2_df['y']-r2_df['y_hat'])
r2_df['y_2'] = r2_df['y']**2
r2_df['dif2'] = r2_df['difference']**2
y_2 = r2_df['y_2'].sum() 
dif2 = r2_df['dif2'].sum()
print(1 - (dif2 / y_2))

0.006583800648086169


In [49]:
rf = pd.DataFrame(rf_predict)
rf.nunique()

0    269351
dtype: int64

In [50]:
pickle.dump(rf_predict, open('rf_predict.pkl', 'wb' ))